<a href="https://colab.research.google.com/github/nisarahamedk/kaggle-riid/blob/master/notebooks/RIID_TF_Transformer_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### RIID TF Transformer Inference

In [2]:
%%capture
!rm -f /opt/conda/lib/python3.7/site-packages/llvmlite-0.31.0-py3.7.egg-info
!pip install ../input/sparse-package-1/numpy-1.19.4-cp37-cp37m-manylinux2010_x86_64.whl
!pip install ../input/sparse-package-1/setuptools-51.1.1-py3-none-any.whl
!pip install ../input/sparse-package-1/scipy-1.6.0-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/sparse-package-1/llvmlite-0.35.0-cp37-cp37m-manylinux2010_x86_64.whl
!pip install ../input/sparse-package-1/numba-0.52.0-cp37-cp37m-manylinux2014_x86_64.whl
!pip install ../input/sparse-package-1/sparse-0.11.2-py2.py3-none-any.whl

Processing /kaggle/input/sparse-package-1/numpy-1.19.4-cp37-cp37m-manylinux2010_x86_64.whl
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.48.0 requires llvmlite<0.32.0,>=0.31.0dev0, which is not installed.
tensorflow 2.3.1 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.4 which is incompatible.
bokeh 2.2.3 requires tornado>=5.1, but you have tornado 5.0.2 which is incompatible.
Processing /kaggle/input/sparse-package-1/setuptools-51.1.1-py3-none-any.whl
  Attempting uninstall: setuptools
    Found existing installation: setuptools 46.1.3.post20200325
    Uninstalling setuptools-46.1.3.post20200325:
      Successfully uninstalled setuptools-46.1.3.post20200325
ERROR: pip's dependency resolver does

In [1]:
import itertools
import pickle
from datetime import datetime

import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
# import sparse

np.random.seed(42)
tf.random.set_seed(42)

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

### Model

#### Positional Encoding

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

#### Look ahead mask¶


In [ ]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

#### Scaled Dot Product Attention

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

#### Multi Head Attention

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

#### Pointwise FeedForward Network

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

#### EncoderLayer

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

#### Encoder

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.content_id_emb = tf.keras.layers.Embedding(embed_size_dict["content_id"] + 2, d_model)
    self.task_container_id_emb = tf.keras.layers.Embedding(embed_size_dict["task_container_id"] + 2, d_model)
    self.part_emb = tf.keras.layers.Embedding(embed_size_dict["part"] + 2, d_model)
    self.prior_question_elapsed_time_emb = tf.keras.layers.Dense(d_model, use_bias=True)
    self.prev_answered_emb = tf.keras.layers.Embedding(4, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)


    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embeddings and position encoding.
    c_emb = self.content_id_emb(x[..., 0])  # (batch_size, input_seq_len, d_model)
    t_emb = self.task_container_id_emb(x[..., 1])
    prior_time_emb = self.prior_question_elapsed_time_emb(tf.expand_dims(x[..., 2], axis=-1))
    pt_emb = self.part_emb(x[..., 3])
    pv_emb = self.prev_answered_emb(x[..., 4])
    x = c_emb + t_emb + prior_time_emb + pt_emb + pv_emb
    
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask) # (batch_size, input_seq_len, d_model)

    return x

In [ ]:
class TransformerSeq2SeqClassifier(keras.models.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate=0.1):
    super(TransformerSeq2SeqClassifier, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, maximum_position_encoding, embed_size_dict, rate)
    self.out = tf.keras.layers.Dense(1, activation="sigmoid")

  def call(self, x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = create_look_ahead_mask(seq_len)
    encoded = self.encoder(x, mask=look_ahead_mask)

    out = self.out(encoded)
    return out # [batch_size, input_seq_len, 1]

#### Embedding sizes

In [ ]:
# DATA_PATH = 'gs://kds-f48a9c4d95386273c0ef508e337abd3f874b82a454a6c3d0e035839c'
DATA_PATH = "/kaggle/input/riid-0-1"

In [ ]:
embed_sizes = pickle.loads(tf.io.read_file(DATA_PATH + "/emb_sz.pkl").numpy())
embed_sizes

In [ ]:
model = TransformerSeq2SeqClassifier(
      num_layers=1,
      d_model=512,
      num_heads=8,
      dff=1024,
      maximum_position_encoding=128,
      embed_size_dict=embed_sizes
  )
model.build(input_shape=(128, 128, 5)) # input_shape - [batch_size, seq_len, features]
model.summary()

In [ ]:
model.load_weights("/kaggle/input/riid-model-2/best-model.h5")

### Dataset

#### Question df

In [2]:
questions_df = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/questions.csv", index_col="question_id")

#### Test API

In [3]:
import riiideducation

# You can only call make_env() once, so don't lose it!
env = riiideducation.make_env()

In [4]:
# You can only iterate through a result from `env.iter_test()` once
# so be careful not to lose it once you start iterating.
iter_test = env.iter_test()

In [5]:
dtypes_train = {
    'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'
    }

dtypes_questions = {
    "question_id": "",
    "bundle_id": "",
    "correct_answer": "",
    "part": "int16",
    "tags": "",
}

In [6]:
SEQ_LEN = 512

#### Submission

#### Profile

In [7]:
# def time(func):
#     def wrapped(*args, **kwargs):
#         t0 = datetime.now()
        
#         ret = func(*args, **kwargs)
        
#         dt = datetime.now() - t0
#         print(f"==> {func.__name__} took {dt.total_seconds():.2} sec")    
#         return ret
#     return wrapped

# @time
def preprocess(test_dfm, prev_test_df):
    
    # we have prev df's answers, so store the prev_test_df in state_dict
    if prev_test_df is not None:
        try:
            prev_test_df['answered_correctly'] = list(filter(lambda x: x !=-1, eval(test_dfm['prior_group_answers_correct'].iloc[0])))
            prev_test_df['answered_correctly'] += 1
            prev_user_group = prev_test_df.groupby("user_id").apply(
                lambda row: row.values[:, 1:] # exclude user_id
            )
            for user in prev_user_group.index:
                if state_dict.get(user, None) is None: # new user, add to the state dict.
                    state_dict[user] = prev_user_group[user]
                else: # existing user
                    state = np.vstack([state_dict[user], prev_user_group[user]]) # append to prev features.
                    if state.shape[0] > SEQ_LEN-1: # we dont need history beyond SEQ_LEN-1
                        state = state[-(SEQ_LEN-1):, :]
                    state_dict[user] = state
        except Exception as e:
            pass # do not update the state dict.
        
    # dummy "answered_correctly" - fill token
    test_dfm["answered_correctly"] = 2
    
    # join question for feaures
    test_df = test_dfm.join(questions_df, on="content_id")
    
    # -- process the current df.
    # selecting required cols
    test_df = test_df[["user_id", "timestamp", "content_id", "task_container_id", "prior_question_elapsed_time", "prior_question_had_explanation", "part", "tags", "answered_correctly"]]
    
    # 0, 1, 2 are special tokens, so increment 3
    indicator_cols = ["content_id", "task_container_id", "part", "prior_question_had_explanation"]
    for c in indicator_cols:
      test_df[c] = test_df[c] + 3
        
    # same treatment for the tags
    test_df["tags"] = test_df["tags"].apply(lambda row: " ".join([str(int(x)+3) for x in row.split(" ")]))
    
    # FIXME: unseen ids - content_id > 13525, 'task_container_id'> 10002, 'part'> 10
    test_df.loc[test_df["content_id"] > 13525, "content_id"] = 0
    test_df.loc[test_df["task_container_id"] > 10002, "task_container_id"] = 0
    test_df.loc[test_df["part"] > 10, "part"] = 0
    # test_df.fillna(2, inplace=True) # FILL TOKEN
    
    return test_df

# @time
def get_x(test_df):
    max_seq_len = 1
    xb = []
    for row in test_df.values:
        user = row[0]
        x = np.zeros((SEQ_LEN+1, 8))
        if state_dict.get(user, None) is None: # new user
            x[-2, :] = [1] * 8 # start token
            x[-1, :] = row[1:] # [1:] - skip user_id
        else: # existing user, get prev states and build time series.
            prev_state = state_dict[user]
            row = np.append(row, 3) # this will be rolled to the first pos
            curr_state = np.vstack([prev_state, row[1:]]) # [1:] - skip user_id

            seq_len = curr_state.shape[0]
            max_seq_len = max(seq_len, max_seq_len)

            x[-seq_len:, :-1] = curr_state[:, :-1] # everything except prev_answer
            x[-seq_len:, -1] = np.roll(curr_state[:, -1], shift=1) # rolled answer as prev_answer
        xb.append(x)
    x = np.stack(xb, axis=0)[:, -max_seq_len:, :]
    return x

# @time
def predict(x):
    
    # predict
    preds = model(x, training=False)[:, -1, :].numpy().flatten()
    # preds = model.predict(x)[:, -1, :].flatten()
    
    return preds

start = datetime.now()
# test_dfs = [] # for debug

prev_test_df = None

# Loading training user states
state_dict = {}
# with open("/kaggle/input/riid-state-dict-512-0-1/state_dict.pkl", "rb") as f:
#     state_dict = pickle.load(f)

for test_dfm, sample_pred in iter_test:
#     print("\n*\n*\n--- iteration ---")
#     t0 = datetime.now()
    
    # filtering only questions, removing lectures.
    test_dfm = test_dfm[test_dfm.content_type_id == False]
    test_df = preprocess(test_dfm, prev_test_df)
    break
#     test_dfs.append(test_df.copy())
    
    prev_test_df = test_df.copy()
    
    x = get_x(test_df)
    
    final_preds = predict(x)
    
    
    # submit
    test_dfm["answered_correctly"] = final_preds
    env.predict(test_dfm[["row_id", "answered_correctly"]])
#     print(f"iteration took: {(datetime.now() - t0).total_seconds():.3}s")
    
total_time = datetime.now() - start
print(f"===== Total time: {total_time.total_seconds():.3} sec")

===== Total time: 0.064 sec


In [8]:
test_dfm

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
group_num,,,,,,,,,,
0,0,0,275030867,5729,0,0,NaN,<NA>,[],[]
0,1,13309898705,554169193,12010,0,4427,19000.0,True,NaN,NaN
0,2,4213672059,1720860329,457,0,240,17000.0,True,NaN,NaN
0,3,62798072960,288641214,13262,0,266,23000.0,True,NaN,NaN
0,4,10585422061,1728340777,6119,0,162,72400.0,True,NaN,NaN
0,5,18020362258,1364159702,12023,0,4424,18000.0,True,NaN,NaN
0,6,2325432079,1521618396,574,0,1367,18000.0,True,NaN,NaN
0,7,39456940781,1317245193,12043,0,5314,17000.0,True,NaN,NaN
0,8,3460555189,1700555100,7910,0,532,21000.0,True,NaN,NaN


In [8]:
test_df

,user_id,timestamp,content_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,part,tags
group_num,,,,,,,,
0,275030867,0,5732,3,NaN,<NA>,8,11
0,554169193,13309898705,12013,4430,19000.0,4,5,93 103 95 32
0,1720860329,4213672059,460,243,17000.0,4,5,146 108 41 32
0,288641214,62798072960,13265,269,23000.0,4,8,99
0,1728340777,10585422061,6122,165,72400.0,4,8,57
0,1364159702,18020362258,12026,4427,18000.0,4,5,5 110 95 32
0,1521618396,2325432079,577,1370,18000.0,4,5,146 144 41 32
0,1317245193,39456940781,12046,5317,17000.0,4,5,146 117 41 32
0,1700555100,3460555189,7913,535,21000.0,4,4,13 181 95


Done.

In [4]:
int64_feat = sparse.load_npz("/kaggle/input/riid-state-dict-512-0-7/int64_feat.npz").todense()

In [5]:
int32_feat = sparse.load_npz("/kaggle/input/riid-state-dict-512-0-7/int32_feat.npz").todense()

In [6]:
int16_feat = sparse.load_npz("/kaggle/input/riid-state-dict-512-0-7/int16_feat.npz").todense()

In [7]:
int8_feat = sparse.load_npz("/kaggle/input/riid-state-dict-512-0-7/int8_feat.npz").todense()

In [8]:
user_index = dict()
with open("/kaggle/input/riid-state-dict-512-0-7/user_index.pkl", "rb") as f:
    user_index = pickle.load(f)

In [9]:
tags_feat = sparse.load_npz("/kaggle/input/riid-state-dict-512-0-7/tags_feat.npz")

In [10]:
tags_feat[user_index[115]].todense()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)